#**주차 수요 분석**
# 단계1 : 데이터 전처리

## 0.미션

단지별 등록 차량 수에 영향을 미치는 요인을 분석하고자 합니다. 이에 맞게 다음의 미션을 수행하시오.
* 미션1 : 데이터 탐색하기
    * 주어진 데이터를 살펴보면서 내용을 파악합니다.
    * Target을 정의하고, 적절한 가설을 도출합니다.
* 미션2 : 데이터 종류에 따른 전처리
    * NaN을 검토하고, 조치합니다. 여기서는 준공일자(준공연도)에 대해서만 처리합니다.
    * 불필요한 칼럼을 제거합니다.
    * 분석 단위(행)를 어떻게 구성할지 고민합니다.
* 미션3 : 세부 데이터 집계 및 결합
    * 단지별 데이터와 상세 데이터를 분리하고
    * 각각에 맞는 전처리를 수행합니다.
    * 가설에 도출된 x변수를 도출하고 하나의 데이터셋으로 통합합니다.

## 1.환경설정

* 세부 요구사항
    * 제공된 압축파일을 다운받아 압축을 풀고
    * anaconda의 root directory(보통 C:/Users/< ID > 에 project2 폴더를 만들고, 복사해 넣습니다.
    * 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### (1) 라이브러리 불러오기

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

### (2) 한글폰트 지정

In [9]:
import matplotlib.font_manager as fm

# 설치된 폰트
font_list = [font.name for font in fm.fontManager.ttflist]

# 맑은고딕이 있는지 확인
'Malgun Gothic' in font_list

True

In [10]:
plt.rcParams['font.family'] = 'Malgun Gothic'

### (3) 경로 설정

* project2 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [13]:
path = ''

### (4) 데이터 불러오기

* 주어진 데이터셋
    * data.xlsx
    * pd.read_excel 을 이용해서 로딩합니다.

In [16]:
file = 'data.xlsx'

In [17]:
data = pd.read_excel(path + file)

## 2.미션1 : 기본 탐색

* **세부 요구사항**
    * 기본 정보 조회 : 상위 n개 조회, information, NaN 조회하기
    * 데이터 탐색하기
        * 기초통계량을 통해 각 값들의 대략의 분포를 살펴봅시다.
        * 전체 단지 수 count
        * 범주 종류 파악하기
        * 그 외 추가 탐색 하기
    * 분석 단위 결정하기
        * 주어진 데이터의 행의 의미 파악, 분석을 위한 단위 검토
    * 가설 도출
        * Target을 정의합니다.
        * 주어진 데이터를 기반으로 초기가설을 도출합니다.
            * 개인별 : 5개 이상
            * 조별 : 10개 이상

### (1) 기본 정보 조회

In [21]:
#전체 데이터의 행,열 개수 확인
data.shape

(1157, 15)

In [22]:
#전체 데이터의 상위 5개 행 확인
data.head()

단지코드         단지명  총세대수  전용면적별세대수  지역        준공일자 건물형태    난방방식 승강기설치여부  \
0  C0001  엘에이치 서초4단지    78        35  서울  20131204.0  계단식  개별가스난방  전체동 설치   
1  C0001  엘에이치 서초4단지    78        43  서울  20131204.0  계단식  개별가스난방  전체동 설치   
2  C0002     LH삼성아파트    35        26  서울  20130801.0  복도식  개별가스난방  전체동 설치   
3  C0002     LH삼성아파트    35         9  서울  20130801.0  복도식  개별가스난방  전체동 설치   
4  C0003     강남LH8단지    88         7  서울  20131023.0  계단식  개별가스난방  전체동 설치   

   단지내주차면수   전용면적  공급면적(공용)     임대보증금     임대료  실차량수  
0      120  51.89   19.2603  50758000  620370   109  
1      120  59.93   22.2446  63166000  665490   109  
2       47  27.75   16.5375  63062000  458640    35  
3       47  29.08   17.3302  63062000  481560    35  
4      106  59.47   21.9462  72190000  586540    88

In [23]:
#전체 데이터의 모든 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      1157 non-null   object 
 1   단지명       1157 non-null   object 
 2   총세대수      1157 non-null   int64  
 3   전용면적별세대수  1157 non-null   int64  
 4   지역        1157 non-null   object 
 5   준공일자      1103 non-null   float64
 6   건물형태      1135 non-null   object 
 7   난방방식      1082 non-null   object 
 8   승강기설치여부   1059 non-null   object 
 9   단지내주차면수   1157 non-null   int64  
 10  전용면적      1157 non-null   float64
 11  공급면적(공용)  1157 non-null   float64
 12  임대보증금     1157 non-null   int64  
 13  임대료       1157 non-null   int64  
 14  실차량수      1157 non-null   int64  
dtypes: float64(3), int64(6), object(6)
memory usage: 135.7+ KB


In [24]:
# NaN 조사
data.isnull().sum()

단지코드         0
단지명          0
총세대수         0
전용면적별세대수     0
지역           0
준공일자        54
건물형태        22
난방방식        75
승강기설치여부     98
단지내주차면수      0
전용면적         0
공급면적(공용)     0
임대보증금        0
임대료          0
실차량수         0
dtype: int64

### (2) 데이터 탐색하기
* **세부 요구사항**
    * 기초통계량을 통해 각 값들의 대략의 분포를 살펴봅시다.
    * 주어진 데이터의 행의 의미 파악, 분석을 위한 단위 검토
    * 전체 단지 수 count
    * 범주 종류 파악하기
    * 그 외 추가 탐색 하기

In [26]:
#전체 데이터의 수치형 변수 분포 확인
data.describe()

총세대수     전용면적별세대수          준공일자      단지내주차면수         전용면적  \
count  1157.000000  1157.000000  1.103000e+03  1157.000000  1157.000000   
mean    659.075194   163.691443  1.973918e+07   682.261884    51.565584   
std     456.110643   166.766358  2.392214e+06   473.331805    18.243315   
min       1.000000     1.000000  2.002000e+03    10.000000    17.590000   
25%     315.000000    44.000000  2.004052e+07   308.000000    39.480000   
50%     595.000000   112.000000  2.009102e+07   629.000000    46.900000   
75%     918.000000   229.000000  2.013121e+07   911.000000    59.810000   
max    2289.000000  1258.000000  2.022071e+07  4553.000000   139.350000   

          공급면적(공용)         임대보증금           임대료         실차량수  
count  1157.000000  1.157000e+03  1.157000e+03  1157.000000  
mean     20.562360  2.850789e+07  2.259409e+05   650.762316  
std       5.164405  2.890687e+07  1.768102e+05   390.573462  
min       5.850000  0.000000e+00  0.000000e+00    21.000000  
25%      16.997400  1.379700e+07  1.177400e+05   320.000000  
50%      20.384700  1.997300e+07  1.842900e+05   626.000000  
75%      23.722500  3.375300e+07  2.634400e+05   894.000000  
max      42.760000  2.549220e+08  1.058030e+06  1657.000000

In [27]:
# 단지 수
data['단지코드'].drop_duplicates().count()

345

In [28]:
# 범주별 종류 파악하기
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      1157 non-null   object 
 1   단지명       1157 non-null   object 
 2   총세대수      1157 non-null   int64  
 3   전용면적별세대수  1157 non-null   int64  
 4   지역        1157 non-null   object 
 5   준공일자      1103 non-null   float64
 6   건물형태      1135 non-null   object 
 7   난방방식      1082 non-null   object 
 8   승강기설치여부   1059 non-null   object 
 9   단지내주차면수   1157 non-null   int64  
 10  전용면적      1157 non-null   float64
 11  공급면적(공용)  1157 non-null   float64
 12  임대보증금     1157 non-null   int64  
 13  임대료       1157 non-null   int64  
 14  실차량수      1157 non-null   int64  
dtypes: float64(3), int64(6), object(6)
memory usage: 135.7+ KB


### (3) 분석 단위

In [30]:
# 분석단위는 어떻게 결정하는게 적절할까요?
# 단지별!

## 2.미션2 : 데이터 종류에 따른 전처리

* **세부 요구사항**
    * NaN 조치
        * NaN를 다시 한번 조회합니다.
        * 준공일자만 여기서 조치합시다.
            * 준공연도만 필요합니다. 문자열로 변환한 후 앞 4자리를 잘라 준공연도로 변환합니다.
            * 잘못 들어간 데이터들과 nan 문자를 준공연도 최소값으로 치환합니다.
        * 나머지 변수들은 탐색 이후에 조치를 취합니다.
    * (필요시) 범주형 변수 통합 : 너무 많은 수의 범주를 적은 수로 통합합니다.
    * 불필요한 데이터 제거


In [33]:
# 전처리 전에 데이터셋 복사
df = data.copy()

### (1) NaN 조치

In [35]:
# 준공일자에서 준공 연도 추출하기

## 준공일자를 문자열로 변환(.astype('str')하고, 앞에서 4자리를 잘라서(.str.slice()) 저장
df['준공연도'] = df['준공일자'].astype('str')
df['준공연도'] = df['준공연도'].str.slice(0,4)

## NaN에 대한 조치 : 삭제? 채우기?

# 변경할 준공연도를 딕셔너리로 정의
change_dict = {'C0077': '2000', 'C0078': '2008', 'C0080': '2015',
               'C0277': '2012', 'C0280': '2009', 'C0281': '2009', 
               'C0282': '2010', 'C0283': '2010', 'C0294': '2005',
               'C0282': '2010', 'C0283': '2010', 'C0294': '2005',
               'C0295': '2010', 'C0301': '2015', 'C0330': '1999',
               'C0340': '2008', 'C1301': '2015', 'C0278': '2010',
               'C0279': '2010', 'C0156': '2008'}
 
# map 함수를 사용하여 값 변경
df['준공연도'] = df['단지코드'].map(change_dict).fillna(df['준공연도'])
 
df['준공연도'] = df['준공연도'].astype('int64')

## 준공 연도를 다시 int로 변환(.astype('int'))
df['준공연도'] = df['준공연도'].astype('int64')


In [36]:
df['준공연도'].describe()

count    1157.000000
mean     2008.437338
std         7.039478
min      1970.000000
25%      2005.000000
50%      2010.000000
75%      2013.000000
max      2022.000000
Name: 준공연도, dtype: float64

### (2) 불필요한 칼럼 제거

* **세부 요구사항**
    * 단지명 : 분석단위로 볼때 단일값.
    * 단지내주차면수 :
        * 본 문제는 등록차량수를 예측하고, 그것을 기반으로 주차면수를 정하는 것이 목적 입니다.
        * 그런데, 역으로 주차면수 먼저 정한후 그것을 기반으로 등록차량수를 예측하는 것은 성립될 수 없습니다.
    * 준공일자 : 준공연도를 사용할 것이므로, 준공일자는 제거

* 불필요한 정보 제거하기

In [40]:
drop_columns = ['단지명', '단지내주차면수', '준공일자']

In [41]:
df = df.drop(columns = drop_columns)

In [42]:
df.head(1)

단지코드  총세대수  전용면적별세대수  지역 건물형태    난방방식 승강기설치여부   전용면적  공급면적(공용)     임대보증금  \
0  C0001    78        35  서울  계단식  개별가스난방  전체동 설치  51.89   19.2603  50758000   

      임대료  실차량수  준공연도  
0  620370   109  2013

## 3.미션3 : 세부 데이터 집계 및 결합

* **세부 요구사항**

    * 데이터프레임을 두가지 형태로 나눕니다.
        * **단지별 데이터** : 중복행 제거
        * **단지내 상세 데이터** : 집계하기
    * 단지별 데이터와 상세 데이터 집계 결과 merge 시키기



### (1) 데이터프레임 두개로 나누기

* **세부 요구사항**
    * 단지별 데이터
        * 단지코드, 총세대수, 지역, 준공일자(준공연도), 건물형태, 난방방식, 승강기설치여부, 실차량수
        * 단지별 데이터를 분할 한 후, 중복행을 제거합니다.
        * 중복행이 잘 제거 되었는지 확인합니다.
    * 상세 데이터
        * 단지코드, 전용면적별세대수,전용면적, 공급면적(공용),임대보증금,임대료

In [47]:
# 단지별 데이터
df1 = df[['단지코드', '총세대수', '지역', '준공연도', '건물형태', '난방방식', '승강기설치여부', '실차량수']].copy()

# 상세 데이터
df2 = df[['단지코드', '전용면적별세대수','전용면적', '공급면적(공용)', '임대보증금','임대료']].copy() # 상세 데이터

In [48]:
# 데이터프레임에서 중복행을 제거하기 : df.drop_duplicates()
df1.drop_duplicates(inplace =True)
df1.shape

(345, 8)

In [49]:
# 확인 : 단지 코드가 유니크해야 함.
unique_codes = df1['단지코드'].unique()
 
# 유일한 값의 개수와 전체 데이터 개수 비교
if len(unique_codes) == len(df1):
    print("모든 단지코드가 유니크합니다.")
else:
    print("중복되는 단지코드가 있습니다.")

모든 단지코드가 유니크합니다.


* 상세 데이터

In [51]:
df2.head()

단지코드  전용면적별세대수   전용면적  공급면적(공용)     임대보증금     임대료
0  C0001        35  51.89   19.2603  50758000  620370
1  C0001        43  59.93   22.2446  63166000  665490
2  C0002        26  27.75   16.5375  63062000  458640
3  C0002         9  29.08   17.3302  63062000  481560
4  C0003         7  59.47   21.9462  72190000  586540

### (2) 상세 데이터 집계하기

* **세부 요구사항**
    * 전용면적 구간으로 나누기(원래, 단변량 분석 후 구간 나누기를 결정하지만, 여기서는 현 미션에서 수행합니다.)
        * 전용면적을 의미 있는 단위로 자른 칼럼 만들기 (pd.cut)
        * 구간별 세대 수 집계
        * pivot
    * 임대보증금 혹은 임대료 : 적절하게 집계하기.
        * 평균, 가중평균, 중앙값 중에서 고려하기.
    * 전체 면적을 계산합니다.(공급면적 * 세대수)
    * 그 외 가설에서 도출된 변수를 생성해 봅시다.

#### 1) 전용면적

* 전용면적을 의미있는 단위로 자른 칼럼 만들기 (pd.cut)

In [56]:
# 단지내 전용면적별 총면적 계산('전용면적' * '전용면적별세대수')

df2['전용면적별총면적'] = df2['전용면적'] * df['전용면적별세대수']
df2.head()

단지코드  전용면적별세대수   전용면적  공급면적(공용)     임대보증금     임대료  전용면적별총면적
0  C0001        35  51.89   19.2603  50758000  620370   1816.15
1  C0001        43  59.93   22.2446  63166000  665490   2576.99
2  C0002        26  27.75   16.5375  63062000  458640    721.50
3  C0002         9  29.08   17.3302  63062000  481560    261.72
4  C0003         7  59.47   21.9462  72190000  586540    416.29

In [57]:
# 전용면적을 구간별로 자르기

bin = [10, 30, 40, 50, 60, np.inf]
label = ['10_30', '30_40', '40_50', '50_60', 'over_60']
df2['전용면적구간'] = pd.cut(df2['전용면적'], bins = bin, labels = label)
df2.head()

단지코드  전용면적별세대수   전용면적  공급면적(공용)     임대보증금     임대료  전용면적별총면적 전용면적구간
0  C0001        35  51.89   19.2603  50758000  620370   1816.15  50_60
1  C0001        43  59.93   22.2446  63166000  665490   2576.99  50_60
2  C0002        26  27.75   16.5375  63062000  458640    721.50  10_30
3  C0002         9  29.08   17.3302  63062000  481560    261.72  10_30
4  C0003         7  59.47   21.9462  72190000  586540    416.29  50_60

In [58]:
# # 다른 구간으로 자르기
# # 주석 풀면 사용 가능

# # 한국 부동산 시장에서 자주 사용되는 전용면적 구간
# bins = [0, 40, 60, 85, np.inf]
# labels = ['S', 'SM', 'M', 'L']
 
# df2['전용면적_구간'] = pd.cut(df2['전용면적'], bins=bins, labels=labels, right=False)
 
# # 결과 확인
# print(df2['전용면적_구간'].value_counts())
# print(len(df2[['단지코드','전용면적_구간']]))

* 단위 칼럼별 집계

In [60]:
tmp = df2.groupby(['단지코드', '전용면적구간'], as_index = False)['전용면적별세대수'].sum()

# 확인
tmp

C:\Users\User\AppData\Local\Temp\ipykernel_16120\3119383772.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tmp = df2.groupby(['단지코드', '전용면적구간'], as_index = False)['전용면적별세대수'].sum()


단지코드   전용면적구간  전용면적별세대수
0     C0001    10_30         0
1     C0001    30_40         0
2     C0001    40_50         0
3     C0001    50_60        78
4     C0001  over_60         0
...     ...      ...       ...
1720  C2349    10_30        40
1721  C2349    30_40         0
1722  C2349    40_50         0
1723  C2349    50_60         0
1724  C2349  over_60         0

[1725 rows x 3 columns]

* pivot

In [62]:
pivot_table = tmp.pivot(index = '단지코드', columns = '전용면적구간', values = '전용면적별세대수')
pivot_table.reset_index(inplace=True)

# 확인
pivot_table

전용면적구간   단지코드  10_30  30_40  40_50  50_60  over_60
0       C0001      0      0      0     78        0
1       C0002     35      0      0      0        0
2       C0003      0      0      0     88        0
3       C0004      0      0      0    150      327
4       C0006     15      0      0      0        0
..        ...    ...    ...    ...    ...      ...
340     C1341    140      0      0      0        0
341     C1354   1071    298      0     17        0
342     C2307    196      0      0      0        0
343     C2343     80      0      0      0        0
344     C2349     40      0      0      0        0

[345 rows x 6 columns]

#### 2) 임대보증금, 임대료 집계 하기(평균)

In [64]:
# 단지코드별 임대보증금과, 임대료 평균
rent_code =  df2.groupby('단지코드', as_index = False)[['임대보증금', '임대료']].mean() 

# 확인
rent_code.head()

단지코드         임대보증금            임대료
0  C0001  5.696200e+07  642930.000000
1  C0002  6.306200e+07  470100.000000
2  C0003  7.219000e+07  586540.000000
3  C0004  1.015167e+08  950305.000000
4  C0006  5.522750e+07  340148.333333

#### 3) 단지별 총 면적 구하기

In [66]:
df2['총면적'] = df2['전용면적'] * df2['전용면적별세대수']

# 확인
df2

단지코드  전용면적별세대수   전용면적  공급면적(공용)     임대보증금     임대료  전용면적별총면적 전용면적구간  \
0     C0001        35  51.89   19.2603  50758000  620370   1816.15  50_60   
1     C0001        43  59.93   22.2446  63166000  665490   2576.99  50_60   
2     C0002        26  27.75   16.5375  63062000  458640    721.50  10_30   
3     C0002         9  29.08   17.3302  63062000  481560    261.72  10_30   
4     C0003         7  59.47   21.9462  72190000  586540    416.29  50_60   
...     ...       ...    ...       ...       ...     ...       ...    ...   
1152  C0356       956  26.37   12.7500   9931000  134540  25209.72  10_30   
1153  C0358        66  24.83   15.1557   2129000   42350   1638.78  10_30   
1154  C0358        54  33.84   20.6553   2902000   57730   1827.36  30_40   
1155  C0359       149  26.37   13.3800   7134000  118880   3929.13  10_30   
1156  C0359       298  31.32   13.8500   8122000  131140   9333.36  30_40   

           총면적  
0      1816.15  
1      2576.99  
2       721.50  
3       261.72  
4       416.29  
...        ...  
1152  25209.72  
1153   1638.78  
1154   1827.36  
1155   3929.13  
1156   9333.36  

[1157 rows x 9 columns]

In [67]:
tmp2= df2.groupby(['단지코드'], as_index=False)['총면적'].sum()
tmp2.head()

단지코드       총면적
0  C0001   4393.14
1  C0002    983.22
2  C0003   5244.69
3  C0004  34607.34
4  C0006    309.50

### (3) 합치기

* **세부 요구사항**
    * [단지별 데이터]를 기준으로 상세데이터로 만든 데이터셋을 하나씩 merge 합니다.
    * merge를 사용할 때, **how = 'left', on = '단지코드'** 옵션을 이용합니다.
        * [단지별 데이터]가 기준(left)입니다.

* 단지별 데이터 + 전용면적별 세대수

In [71]:
result1 = pd.merge(df1, tmp2, how = 'left', on = '단지코드')
result1 = pd.merge(result1, pivot_table, how = 'left', on = '단지코드')

# 확인
result1

단지코드  총세대수    지역  준공연도 건물형태    난방방식 승강기설치여부  실차량수       총면적  10_30  \
0    C0001    78    서울  2013  계단식  개별가스난방  전체동 설치   109   4393.14      0   
1    C0002    35    서울  2013  복도식  개별가스난방  전체동 설치    35    983.22     35   
2    C0003    88    서울  2013  계단식  개별가스난방  전체동 설치    88   5244.69      0   
3    C0004   477    서울  2014  복도식    지역난방  전체동 설치   943  34607.34      0   
4    C0006    15    서울  2013  복도식  개별가스난방  전체동 설치    21    309.50     15   
..     ...   ...   ...   ...  ...     ...     ...   ...       ...    ...   
340  C0354  1485  대전충남  1993  복도식    중앙난방  전체동 설치   298  40792.77   1181   
341  C1354  1386  대전충남  1993  복도식  중앙가스난방  전체동 설치   258  38472.21   1071   
342  C0356   956    경기  1994  복도식  지역가스난방  전체동 설치   243  25209.72    956   
343  C0358   120    강원  2020  복도식     NaN  전체동 설치    47   3466.14     66   
344  C0359   447  대구경북  1994  복도식  중앙유류난방  전체동 설치    78  13262.49    149   

     30_40  40_50  50_60  over_60  
0        0      0     78        0  
1        0      0      0        0  
2        0      0     88        0  
3        0      0    150      327  
4        0      0      0        0  
..     ...    ...    ...      ...  
340    298      0      6        0  
341    298      0     17        0  
342      0      0      0        0  
343     54      0      0        0  
344    298      0      0        0  

[345 rows x 14 columns]

* 평균 임대 보증금/임대료 합치기

In [73]:
result2 = pd.merge(result1, rent_code, how = 'left', on = '단지코드')

# 확인
result2

단지코드  총세대수    지역  준공연도 건물형태    난방방식 승강기설치여부  실차량수       총면적  10_30  \
0    C0001    78    서울  2013  계단식  개별가스난방  전체동 설치   109   4393.14      0   
1    C0002    35    서울  2013  복도식  개별가스난방  전체동 설치    35    983.22     35   
2    C0003    88    서울  2013  계단식  개별가스난방  전체동 설치    88   5244.69      0   
3    C0004   477    서울  2014  복도식    지역난방  전체동 설치   943  34607.34      0   
4    C0006    15    서울  2013  복도식  개별가스난방  전체동 설치    21    309.50     15   
..     ...   ...   ...   ...  ...     ...     ...   ...       ...    ...   
340  C0354  1485  대전충남  1993  복도식    중앙난방  전체동 설치   298  40792.77   1181   
341  C1354  1386  대전충남  1993  복도식  중앙가스난방  전체동 설치   258  38472.21   1071   
342  C0356   956    경기  1994  복도식  지역가스난방  전체동 설치   243  25209.72    956   
343  C0358   120    강원  2020  복도식     NaN  전체동 설치    47   3466.14     66   
344  C0359   447  대구경북  1994  복도식  중앙유류난방  전체동 설치    78  13262.49    149   

     30_40  40_50  50_60  over_60         임대보증금            임대료  
0        0      0     78        0  5.696200e+07  642930.000000  
1        0      0      0        0  6.306200e+07  470100.000000  
2        0      0     88        0  7.219000e+07  586540.000000  
3        0      0    150      327  1.015167e+08  950305.000000  
4        0      0      0        0  5.522750e+07  340148.333333  
..     ...    ...    ...      ...           ...            ...  
340    298      0      6        0  7.595571e+06  104975.714286  
341    298      0     17        0  8.092875e+06  111848.750000  
342      0      0      0        0  9.931000e+06  134540.000000  
343     54      0      0        0  2.515500e+06   50040.000000  
344    298      0      0        0  7.628000e+06  125010.000000  

[345 rows x 16 columns]

### (4) 추가 변수 만들기
* 가설에서 도출된 변수를 추가로 도출하고, 기존 데이터셋에 추가해 봅시다.

## 4.데이터셋 저장하기

* **세부 요구사항**
    * joblib.dump를 이용하시오.
    * 저장할 파일의 확장자는 보통 .pkl 입니다.

In [77]:
import joblib
joblib.dump(result2, 'result_df.pkl')

['result_df.pkl']

In [78]:
data2 = joblib.load('result_df.pkl')
data2

단지코드  총세대수    지역  준공연도 건물형태    난방방식 승강기설치여부  실차량수       총면적  10_30  \
0    C0001    78    서울  2013  계단식  개별가스난방  전체동 설치   109   4393.14      0   
1    C0002    35    서울  2013  복도식  개별가스난방  전체동 설치    35    983.22     35   
2    C0003    88    서울  2013  계단식  개별가스난방  전체동 설치    88   5244.69      0   
3    C0004   477    서울  2014  복도식    지역난방  전체동 설치   943  34607.34      0   
4    C0006    15    서울  2013  복도식  개별가스난방  전체동 설치    21    309.50     15   
..     ...   ...   ...   ...  ...     ...     ...   ...       ...    ...   
340  C0354  1485  대전충남  1993  복도식    중앙난방  전체동 설치   298  40792.77   1181   
341  C1354  1386  대전충남  1993  복도식  중앙가스난방  전체동 설치   258  38472.21   1071   
342  C0356   956    경기  1994  복도식  지역가스난방  전체동 설치   243  25209.72    956   
343  C0358   120    강원  2020  복도식     NaN  전체동 설치    47   3466.14     66   
344  C0359   447  대구경북  1994  복도식  중앙유류난방  전체동 설치    78  13262.49    149   

     30_40  40_50  50_60  over_60         임대보증금            임대료  
0        0      0     78        0  5.696200e+07  642930.000000  
1        0      0      0        0  6.306200e+07  470100.000000  
2        0      0     88        0  7.219000e+07  586540.000000  
3        0      0    150      327  1.015167e+08  950305.000000  
4        0      0      0        0  5.522750e+07  340148.333333  
..     ...    ...    ...      ...           ...            ...  
340    298      0      6        0  7.595571e+06  104975.714286  
341    298      0     17        0  8.092875e+06  111848.750000  
342      0      0      0        0  9.931000e+06  134540.000000  
343     54      0      0        0  2.515500e+06   50040.000000  
344    298      0      0        0  7.628000e+06  125010.000000  

[345 rows x 16 columns]